In [ ]:
%run DOVSOutages.ipynb
%run AMINonVee.ipynb
%run AMINonVeeSQL.ipynb
%run AMINonVeeCircuitSQL.ipynb
%run MeterPremise.ipynb

In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import itertools
import copy
import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Plot_Box_sns
import GrubbsTest

# Investigate differences when calculate_ket_kwh = True vs False in build_sql_outage_others_on_circuit

- Expect some columns to be equal between e.g., [stop_after_agg_1=True,  groupby_xfmr_in_agg_1=False] and [stop_after_agg_1=False,  groupby_xfmr_in_agg_1=True]
- I was seeing some differences specifically for the case of aep_derived_uom='UNK"
- Differences found to be due to my use of UNION instead of UNION ALL.  I was using UNION, which removes duplicates.  However, for the case where stop_after_agg_1=True, no union occurs, therefore no duplicate removal!
- The UNION is more correct.  However, I need to implement more general methods for duplicate removal.

In [ ]:
conn_aws = Utilities.get_athena_prod_aws_connection()

In [ ]:
cols_of_interest_usage_agg = ['starttimeperiod', 'endtimeperiod', 'aep_endtime_utc', 'timezoneoffset', 
                              'aep_derived_uom', 'aep_srvc_qlty_idntfr', 'value', 'aep_usage_dt']

serial_numbers = [884390632, 880682782, 880320285, 880076534, 880320207, 889926007, 884390655, 880076535]
date_range = ['2021-01-01', '2021-01-02']
field_descs=cols_of_interest_usage_agg
agg_cols_and_types = {'U.value':['sum', 'sq_sum', 'mean', 'std', 'count']}
groupby_cols = ['starttimeperiod', 'endtimeperiod', 'aep_endtime_utc', 'timezoneoffset', 
                'aep_derived_uom', 'aep_srvc_qlty_idntfr', 'aep_usage_dt']
try_to_split_col_strs = True

sort_cols = ['aep_endtime_utc', 'aep_derived_uom', 'aep_srvc_qlty_idntfr']

In [ ]:
sql_stmnt_agg_1 = AMINonVeeCircuitSQL.build_sql_outage_others_on_circuit(
    conn_aws, 
    serial_numbers, date_range, 
    field_descs, agg_cols_and_types, groupby_cols, 
    aep_derived_uoms_and_idntfrs=None, 
    calculate_net_kwh=False, addtnl_build_net_kwh_kwargs=None, 
    stop_after_agg_1=True,  groupby_xfmr_in_agg_1=False, 
    addtnl_build_agg_1_sql_kwargs=dict(try_to_split_col_strs=try_to_split_col_strs),
    include_counts_including_null=True, 
    verbose=False
)

In [ ]:
sql_stmnt_agg_1 = AMINonVeeCircuitSQL.build_sql_outage_others_on_circuit(
    conn_aws, 
    serial_numbers, date_range, 
    field_descs, agg_cols_and_types, groupby_cols, 
    aep_derived_uoms_and_idntfrs=None, 
    calculate_net_kwh=False, addtnl_build_net_kwh_kwargs=None, 
    stop_after_agg_1=True,  groupby_xfmr_in_agg_1=False, 
    addtnl_build_agg_1_sql_kwargs=dict(try_to_split_col_strs=try_to_split_col_strs),
    include_counts_including_null=True, 
    verbose=False
)

In [ ]:
sql_stmnt_agg_1_net_kwh = AMINonVeeCircuitSQL.build_sql_outage_others_on_circuit(
    conn_aws, 
    serial_numbers, date_range, 
    field_descs, agg_cols_and_types, groupby_cols, 
    aep_derived_uoms_and_idntfrs=None, 
    calculate_net_kwh=True, addtnl_build_net_kwh_kwargs=None, 
    stop_after_agg_1=True,  groupby_xfmr_in_agg_1=False, 
    addtnl_build_agg_1_sql_kwargs=dict(try_to_split_col_strs=try_to_split_col_strs),
    include_counts_including_null=True, 
    verbose=False
)

In [ ]:
sql_stmnt_agg_1_gpby_xfmr = AMINonVeeCircuitSQL.build_sql_outage_others_on_circuit(
    conn_aws, 
    serial_numbers, date_range, 
    field_descs, agg_cols_and_types, groupby_cols, 
    aep_derived_uoms_and_idntfrs=None, 
    calculate_net_kwh=False, addtnl_build_net_kwh_kwargs=None, 
    stop_after_agg_1=True,  groupby_xfmr_in_agg_1=True, 
    addtnl_build_agg_1_sql_kwargs=dict(try_to_split_col_strs=try_to_split_col_strs), 
    include_counts_including_null=True, 
    verbose=False
)

In [ ]:
sql_stmnt_full = AMINonVeeCircuitSQL.build_sql_outage_others_on_circuit(
    conn_aws, 
    serial_numbers, date_range, 
    field_descs, agg_cols_and_types, groupby_cols, 
    aep_derived_uoms_and_idntfrs=None, 
    calculate_net_kwh=False, addtnl_build_net_kwh_kwargs=None, 
    stop_after_agg_1=False,  groupby_xfmr_in_agg_1=True, 
    addtnl_build_agg_1_sql_kwargs=dict(try_to_split_col_strs=try_to_split_col_strs), 
    include_counts_including_null=True, 
    verbose=False
)

In [ ]:
sql_stmnt_full_net_kwh = AMINonVeeCircuitSQL.build_sql_outage_others_on_circuit(
    conn_aws, 
    serial_numbers, date_range, 
    field_descs, agg_cols_and_types, groupby_cols, 
    aep_derived_uoms_and_idntfrs=None, 
    calculate_net_kwh=True, addtnl_build_net_kwh_kwargs=None, 
    stop_after_agg_1=False,  groupby_xfmr_in_agg_1=True, 
    addtnl_build_agg_1_sql_kwargs=dict(try_to_split_col_strs=try_to_split_col_strs), 
    include_counts_including_null=True, 
    verbose=False
)

In [ ]:
df_agg_1 = pd.read_sql(sql_stmnt_agg_1, conn_aws)
df_agg_1 = clm.remove_table_aliases(df_agg_1)

In [ ]:
df_agg_1_net_kwh = pd.read_sql(sql_stmnt_agg_1_net_kwh, conn_aws)
df_agg_1_net_kwh = clm.remove_table_aliases(df_agg_1_net_kwh)

In [ ]:
df_agg_1_gpby_xfmr = pd.read_sql(sql_stmnt_agg_1_gpby_xfmr, conn_aws)
df_agg_1_gpby_xfmr = clm.remove_table_aliases(df_agg_1_gpby_xfmr)

In [ ]:
df_full = pd.read_sql(sql_stmnt_full, conn_aws)
df_full = clm.remove_table_aliases(df_full)

In [ ]:
df_full_net_kwh = pd.read_sql(sql_stmnt_full_net_kwh, conn_aws)
df_full_net_kwh = clm.remove_table_aliases(df_full_net_kwh)

In [ ]:
usg_sql = """
WITH MP AS (
	SELECT
		mfr_devc_ser_nbr,
		trsf_pole_nb
	FROM default.meter_premise
	WHERE ((circuit_nb = '06' AND circuit_nm = 'F-8906' AND station_nb = '0089' AND station_nm = 'SHANNON') OR trsf_pole_nb IN ('1893161692563','1893355692345','1893751692088','1893772692757','1893899692721','1893951691949','1894151692642','1894214691507','1894621689717','1894659692098','1894664692742','1894678692290','1894768691393','1894943690495','1895049690979','1895178691337','1895190688028','1895221691730','1895235691963','1895246688635','1895255692207','1895272691100','1895277692451','1895297692694','1895370693033','1895419689332','1895538690860','1895591688085','1895653691427','1895669689010','1895673688667','1895705692098','1895765692797','1895787693054','1895802691061','1895967692502','1895973688498','1896012692779','1896047688699','1896061691034','1896071691608','1896142689907','1896309691590','1896315692166','1896334691869','1896447691298','1896518692741','1896666693007','1896729691561','1896762692722','1896787690973','1896900691263','1896951692709','1897157690938','1897169691673','1897408691211','1897446691568','1897490692052','1897699691192','1897720691449','1897741691677','1897865690874','1898038691760','1898058691998','1898320691619','1898348691146','1898361692107','1898448692512','1898542690818','1898602691388','1898660692325','1898754690937','1898929691480','1898946692563','1898957691780','1898973692077','1899011693413','1899106690894','1899182691250','1899186691373','1899251692330','1899262692739','1899306686450','1899330690530','1899371693192','1899546687512','1899603693174','1899614693453','1899649692288','1899871692788','1899904694088','1900092693415','1900189693043','1900446693384','1900478693062','1900535686424','1900618686345','1900748686223','1900748694060','1900817693669','1900894686082','1902544684474','1902790683950','1903623679856','1903780683051','1904182681045','1904650682157','1905083680959','1905578681621','1905640682612','1905661682903','1905687683306','1905736683825','1905757684156','1905864684957','1905982685611','1893219692283','1893264692222','1893536692216','1894149691809','1894280692399','1894403692776','1894449691795','1894481692273','1894560691535','1894742692493','1894906690009','1894969691261','1895207688217','1895226688429','1895304688796','1895333688974','1895360690468','1895581688258','1895635688596','1895638690270','1895648689269','1895663691073','1895680691825','1895722692307','1895755692661','1895767688084','1895864690116','1895886691337','1895899688297','1896050692190','1896074689159','1896105691885','1896194693045','1896209691318','1896220692485','1896263692761','1896274689106','1896306691018','1896429693028','1896437689065','1896452689696','1896539691575','1896556692151','1896563691854','1896594692451','1896682691279','1896700692441','1896755691839','1896806691269','1896893692988','1896896691832','1896918692102','1896933692339','1896967689336','1897146691437','1897178691797','1897186691908','1897227692256','1897279690927','1897317692840','1897328692949','1897466691810','1897624690894','1897762691919','1897796692230','1897854692791','1898025691521','1898058692601','1898077692251','1898120691163','1898340691865','1898434690828','1898619691638','1898629691745','1898649691985','1898906691247','1898955692686','1898963692341','1898975692923','1898990693157','1899245691812','1899246692505','1899276692972','1899367693469','1899471686652','1899526692602','1899566692886','1899637687248','1899833693161','1900175686768','1900213687659','1900318693697','1900512684131','1900539693369','1900609693369','1900833684107','1901141684086','1901816683611','1901933681296','1902975679875','1903015683914','1903015684150','1903140683755','1903272683613','1903445683433','1903718680905','1904750680226','1904973685590','1905424683819','1905632682406','1905741684371','1905774683523','1905785684538','1905800685208','1905835685763','1905912686911','1905954684979','1905956685108','1906067685050'))
),  
U AS (
	SELECT
		serialnumber,
		starttimeperiod,
		endtimeperiod,
		aep_endtime_utc,
		timezoneoffset,
		aep_derived_uom,
		aep_srvc_qlty_idntfr,
		value,
		aep_usage_dt
	FROM usage_nonvee.reading_ivl_nonvee USG_i
		INNER JOIN MP ON USG_i.serialnumber=MP.mfr_devc_ser_nbr
	WHERE aep_usage_dt BETWEEN '2021-01-01' AND '2021-01-02'
) 
SELECT * FROM U
"""

In [ ]:
usg_sql_2 = """
WITH MP AS (
	SELECT
		mfr_devc_ser_nbr,
		trsf_pole_nb
	FROM default.meter_premise
	WHERE ((circuit_nb = '06' AND circuit_nm = 'F-8906' AND station_nb = '0089' AND station_nm = 'SHANNON') OR trsf_pole_nb IN ('1893161692563','1893355692345','1893751692088','1893772692757','1893899692721','1893951691949','1894151692642','1894214691507','1894621689717','1894659692098','1894664692742','1894678692290','1894768691393','1894943690495','1895049690979','1895178691337','1895190688028','1895221691730','1895235691963','1895246688635','1895255692207','1895272691100','1895277692451','1895297692694','1895370693033','1895419689332','1895538690860','1895591688085','1895653691427','1895669689010','1895673688667','1895705692098','1895765692797','1895787693054','1895802691061','1895967692502','1895973688498','1896012692779','1896047688699','1896061691034','1896071691608','1896142689907','1896309691590','1896315692166','1896334691869','1896447691298','1896518692741','1896666693007','1896729691561','1896762692722','1896787690973','1896900691263','1896951692709','1897157690938','1897169691673','1897408691211','1897446691568','1897490692052','1897699691192','1897720691449','1897741691677','1897865690874','1898038691760','1898058691998','1898320691619','1898348691146','1898361692107','1898448692512','1898542690818','1898602691388','1898660692325','1898754690937','1898929691480','1898946692563','1898957691780','1898973692077','1899011693413','1899106690894','1899182691250','1899186691373','1899251692330','1899262692739','1899306686450','1899330690530','1899371693192','1899546687512','1899603693174','1899614693453','1899649692288','1899871692788','1899904694088','1900092693415','1900189693043','1900446693384','1900478693062','1900535686424','1900618686345','1900748686223','1900748694060','1900817693669','1900894686082','1902544684474','1902790683950','1903623679856','1903780683051','1904182681045','1904650682157','1905083680959','1905578681621','1905640682612','1905661682903','1905687683306','1905736683825','1905757684156','1905864684957','1905982685611','1893219692283','1893264692222','1893536692216','1894149691809','1894280692399','1894403692776','1894449691795','1894481692273','1894560691535','1894742692493','1894906690009','1894969691261','1895207688217','1895226688429','1895304688796','1895333688974','1895360690468','1895581688258','1895635688596','1895638690270','1895648689269','1895663691073','1895680691825','1895722692307','1895755692661','1895767688084','1895864690116','1895886691337','1895899688297','1896050692190','1896074689159','1896105691885','1896194693045','1896209691318','1896220692485','1896263692761','1896274689106','1896306691018','1896429693028','1896437689065','1896452689696','1896539691575','1896556692151','1896563691854','1896594692451','1896682691279','1896700692441','1896755691839','1896806691269','1896893692988','1896896691832','1896918692102','1896933692339','1896967689336','1897146691437','1897178691797','1897186691908','1897227692256','1897279690927','1897317692840','1897328692949','1897466691810','1897624690894','1897762691919','1897796692230','1897854692791','1898025691521','1898058692601','1898077692251','1898120691163','1898340691865','1898434690828','1898619691638','1898629691745','1898649691985','1898906691247','1898955692686','1898963692341','1898975692923','1898990693157','1899245691812','1899246692505','1899276692972','1899367693469','1899471686652','1899526692602','1899566692886','1899637687248','1899833693161','1900175686768','1900213687659','1900318693697','1900512684131','1900539693369','1900609693369','1900833684107','1901141684086','1901816683611','1901933681296','1902975679875','1903015683914','1903015684150','1903140683755','1903272683613','1903445683433','1903718680905','1904750680226','1904973685590','1905424683819','1905632682406','1905741684371','1905774683523','1905785684538','1905800685208','1905835685763','1905912686911','1905954684979','1905956685108','1906067685050'))
),  
KWH_USG_DELREC_W_SIGNED_VAL AS (
	SELECT
		serialnumber,
		starttimeperiod,
		endtimeperiod,
		aep_endtime_utc,
		timezoneoffset,
		aep_derived_uom,
		aep_srvc_qlty_idntfr,
		value,
		aep_usage_dt,
		IF(aep_srvc_qlty_idntfr='RECEIVED', -1*value, value) AS signed_value
	FROM usage_nonvee.reading_ivl_nonvee USG_i
		INNER JOIN MP ON USG_i.serialnumber=MP.mfr_devc_ser_nbr
	WHERE aep_usage_dt BETWEEN '2021-01-01' AND '2021-01-02'
	AND   ((aep_derived_uom = 'KWH' AND aep_srvc_qlty_idntfr = 'RECEIVED') OR (aep_derived_uom = 'KWH' AND aep_srvc_qlty_idntfr = 'DELIVERED'))
), 

KWH_USG_DELREC_NET AS (
	SELECT
		serialnumber,
		starttimeperiod,
		endtimeperiod,
		aep_endtime_utc,
		timezoneoffset,
		aep_derived_uom,
		'DEL_MINUS_REC' AS aep_srvc_qlty_idntfr,
		SUM(signed_value) AS value,
		aep_usage_dt
	FROM KWH_USG_DELREC_W_SIGNED_VAL
	GROUP BY
		serialnumber,
		starttimeperiod,
		endtimeperiod,
		aep_endtime_utc,
		timezoneoffset,
		aep_derived_uom,
		aep_usage_dt
	HAVING COUNT(signed_value) = 2
), 

KWH_USG_TOTAL_VAL AS (
	SELECT
		serialnumber,
		starttimeperiod,
		endtimeperiod,
		aep_endtime_utc,
		timezoneoffset,
		aep_derived_uom,
		aep_srvc_qlty_idntfr,
		value,
		aep_usage_dt
	FROM usage_nonvee.reading_ivl_nonvee USG_i
		INNER JOIN MP ON USG_i.serialnumber=MP.mfr_devc_ser_nbr
	WHERE aep_usage_dt BETWEEN '2021-01-01' AND '2021-01-02'
	AND   ((aep_derived_uom = 'KWH' AND aep_srvc_qlty_idntfr = 'TOTAL'))
), 

KWH_USG_DELREC_NET_UNION_TOTAL_0 AS (
	(
	SELECT
		serialnumber,
		starttimeperiod,
		endtimeperiod,
		aep_endtime_utc,
		timezoneoffset,
		aep_derived_uom,
		aep_srvc_qlty_idntfr,
		value,
		aep_usage_dt
	FROM KWH_USG_DELREC_NET
	)
	UNION
	(
	SELECT
		serialnumber,
		starttimeperiod,
		endtimeperiod,
		aep_endtime_utc,
		timezoneoffset,
		aep_derived_uom,
		aep_srvc_qlty_idntfr,
		value,
		aep_usage_dt
	FROM KWH_USG_TOTAL_VAL
	)
), 

KWH_USG_DELREC_NET_UNION_TOTAL AS (
	SELECT
		serialnumber,
		starttimeperiod,
		endtimeperiod,
		aep_endtime_utc,
		timezoneoffset,
		aep_derived_uom,
		'CALCULATED_NET' AS aep_srvc_qlty_idntfr,
		SUM(value) AS value,
		aep_usage_dt
	FROM KWH_USG_DELREC_NET_UNION_TOTAL_0
	GROUP BY
		serialnumber,
		starttimeperiod,
		endtimeperiod,
		aep_endtime_utc,
		timezoneoffset,
		aep_derived_uom,
		aep_usage_dt
	HAVING COUNT(value) = 1
), 

U AS (
	(
	SELECT
		*
	FROM KWH_USG_DELREC_NET_UNION_TOTAL
	)
	UNION
	(
	SELECT
		serialnumber,
		starttimeperiod,
		endtimeperiod,
		aep_endtime_utc,
		timezoneoffset,
		aep_derived_uom,
		aep_srvc_qlty_idntfr,
		value,
		aep_usage_dt
	FROM usage_nonvee.reading_ivl_nonvee USG_i
		INNER JOIN MP ON USG_i.serialnumber=MP.mfr_devc_ser_nbr
	WHERE aep_usage_dt BETWEEN '2021-01-01' AND '2021-01-02'
	AND   aep_derived_uom <> 'KWH'
	)
) 
SELECT * FROM U
"""

In [ ]:
usg_df = pd.read_sql(usg_sql, conn_aws)
usg_df = clm.remove_prepend_from_columns_in_df(usg_df)

In [ ]:
usg_df_2 = pd.read_sql(usg_sql_2, conn_aws)
usg_df_2 = clm.remove_prepend_from_columns_in_df(usg_df_2)

In [ ]:
approx_dfs = Utilities_df.get_dfs_diff_approx_ok(df_agg_1[df_agg_1['aep_derived_uom']!='KWH'], 
                                                 df_agg_1_net_kwh[df_agg_1_net_kwh['aep_derived_uom']!='KWH'], 
                                                 sort_by=sort_cols, 
                                                 return_df_only=True)

In [ ]:
approx_dfs

In [ ]:
usg_df[(usg_df['aep_endtime_utc']=='1609478100') & (usg_df['aep_derived_uom']=='UNK')]

In [ ]:
usg_df_2[(usg_df_2['aep_endtime_utc']=='1609478100') & (usg_df_2['aep_derived_uom']=='UNK')]

In [ ]:
usg_df[(usg_df['aep_endtime_utc']=='1609478100') & (usg_df['aep_derived_uom']=='UNK')].iloc[0]==usg_df[(usg_df['aep_endtime_utc']=='1609478100') & (usg_df['aep_derived_uom']=='UNK')].iloc[1]

In [ ]:
df_agg_1[df_agg_1['aep_derived_uom']!='KWH'].sort_values(by=sort_cols, ignore_index=True).iloc[approx_dfs.index.get_level_values(0).unique().tolist()]

In [ ]:
df_agg_1_net_kwh[df_agg_1_net_kwh['aep_derived_uom']!='KWH'].sort_values(by=sort_cols, ignore_index=True).iloc[approx_dfs.index.get_level_values(0).unique().tolist()]

In [ ]:
# df_agg_1.shape[0] should equal df_full.shape[0]
# df_full_net_kwh.shape[0] should equal df_full.shape[0] minus 2/3 the number of
#   entries with 'aep_derived_uom'=='KWH', as 'RECIEVED', 'DELIVERED', and 'TOTAL' have been
#   compressed into CALCULATED_NET

In [ ]:
print(df_agg_1.shape[0]==df_full.shape[0])
print(df_full_net_kwh.shape[0]== df_full.shape[0] - (2/3)*df_full[df_full['aep_derived_uom']=='KWH'].shape[0])

In [ ]:
print(df_agg_1.shape)
print(df_agg_1_gpby_xfmr.shape)
print(df_full.shape)
print(df_full_net_kwh.shape)

In [ ]:
sort_cols = ['aep_endtime_utc', 'aep_derived_uom', 'aep_srvc_qlty_idntfr']

In [ ]:
df_full[df_full['aep_derived_uom']!='KWH'].sort_values(by=sort_cols, ignore_index=True).equals(df_full_net_kwh[df_full_net_kwh['aep_derived_uom']!='KWH'].sort_values(by=sort_cols, ignore_index=True))

In [ ]:
approx_dfs = Utilities_df.get_dfs_diff_approx_ok(df_full[df_full['aep_derived_uom']!='KWH'], 
                                                 df_full_net_kwh[df_full_net_kwh['aep_derived_uom']!='KWH'], 
                                                 sort_by=sort_cols, 
                                                 return_df_only=True)

In [ ]:
approx_dfs

In [ ]:
# approx_dfs.index.get_level_values(0).unique().tolist()

In [ ]:
df_full[df_full['aep_derived_uom']!='KWH'].sort_values(by=sort_cols, ignore_index=True).iloc[approx_dfs.index.get_level_values(0).unique().tolist()]

In [ ]:
df_full_net_kwh[df_full_net_kwh['aep_derived_uom']!='KWH'].sort_values(by=sort_cols, ignore_index=True).iloc[approx_dfs.index.get_level_values(0).unique().tolist()]

In [ ]:
df_full[df_full['aep_derived_uom']!='KWH'].sort_values(by=sort_cols, ignore_index=True).head()

In [ ]:
df_full_net_kwh[df_full_net_kwh['aep_derived_uom']!='KWH'].sort_values(by=sort_cols, ignore_index=True).head()

In [ ]:
df_agg_1.sort_values(by=['aep_endtime_utc', 'aep_derived_uom', 'aep_srvc_qlty_idntfr']).head()

In [ ]:
df_full.sort_values(by=['aep_endtime_utc', 'aep_derived_uom', 'aep_srvc_qlty_idntfr']).head()

In [ ]:
df_full[(df_full['aep_derived_uom']=='KWH') & 
        ((df_full['aep_srvc_qlty_idntfr']=='RECEIVED') | (df_full['aep_srvc_qlty_idntfr']=='DELIVERED'))]

In [ ]:
df_full[(df_full['aep_derived_uom']=='KWH') & 
        (df_full['aep_srvc_qlty_idntfr']=='TOTAL')]

In [ ]:
df_full_net_kwh[df_full_net_kwh['aep_derived_uom']=='KWH'].shape

In [ ]:
df_full[df_full['aep_derived_uom']=='KWH'].shape

# Investigate weird SNs where KWH TOTAL = KWH RECEIVED
- Began as verification that combining KWH via SQL gave same results as using AMINonVee.combine_kwh_delivered_and_received_values
- However, observed this strange behavior

In [ ]:
cols_of_interest_usage = ['serialnumber', 'starttimeperiod', 'endtimeperiod', 'aep_endtime_utc', 'timezoneoffset',
                          'aep_derived_uom', 'aep_srvc_qlty_idntfr', 'value', 'aep_usage_dt']

cols_of_interest_usage_agg = ['starttimeperiod', 'endtimeperiod', 'aep_endtime_utc', 'timezoneoffset', 
                              'aep_derived_uom', 'aep_srvc_qlty_idntfr', 'value', 'aep_usage_dt']

cols_of_interest_met_prem = ['mfr_devc_ser_nbr', 'prem_nb', 'srvc_pole_nb', 
                             'trsf_pole_nb', 'annual_kwh', 'station_nb', 'xfmr_nb']

date_range = ['2021-01-01', '2021-01-02']
field_descs=cols_of_interest_usage_agg
agg_cols_and_types = {'U.value':['sum', 'sq_sum', 'mean', 'std', 'count']}
groupby_cols = ['starttimeperiod', 'endtimeperiod', 'aep_endtime_utc', 'timezoneoffset', 
                'aep_derived_uom', 'aep_srvc_qlty_idntfr', 'aep_usage_dt']
try_to_split_col_strs = True

tmp_sns = [638995834, 640314914, 640348584, 640348584, 645786659, 681348416, 683596072, 750388945, 
 761934246, 876277766, 876682976, 876683003, 876683004, 879832166, 880064346, 880071670, 
 880081025, 880157481, 880181013, 880184364]

In [ ]:
usg_norm_select = SQLSelect(cols_of_interest_usage)
#-----
usg_norm_where = SQLWhere([dict(field_desc='aep_opco', comparison_operator='=', value='oh', needs_quotes=True), 
                           dict(field_desc='aep_derived_uom', comparison_operator='=', value='KWH', needs_quotes=True), 
                           dict(field_desc='aep_usage_dt', comparison_operator='BETWEEN', 
                                value=[f'{date_range[0]}',f'{date_range[1]}'], needs_quotes=True), 
                           dict(field_desc='serialnumber', comparison_operator='IN', 
                                value=f'({Utilities_sql.join_list_w_quotes(tmp_sns)})', needs_quotes=False)
                          ], idxs=None, run_check=True)
#-----
usg_norm = SQLQuery(sql_select = usg_norm_select, 
                                sql_from = SQLFrom('usage_nonvee', 'reading_ivl_nonvee'), 
                                sql_where = usg_norm_where
                               )
# print(usg_norm.get_sql_statement())
#------------------------------
usg_norm_df = pd.read_sql(usg_norm.get_sql_statement(), conn_aws)
usg_norm_df = clm.remove_table_aliases(usg_norm_df)

In [ ]:
usg_kwh_net_or_total_sql_stmnt = AMINonVeeSQL.build_net_kwh_usage_sql_statement(cols_of_interest_usage, 
                                                                   date_range=date_range, 
                                                                   serial_numbers=tmp_sns, 
                                                                   additional_derived_uoms=None, 
                                                                   run_careful=True, 
                                                                   value_col='value', 
                                                                   aep_srvc_qlty_idntfr_col='aep_srvc_qlty_idntfr', 
                                                                   serialnumber_col='serialnumber', 
                                                                   return_statement=True, 
                                                                   final_table_alias='U', 
                                                                   insert_n_tabs_to_each_line=1, 
                                                                   prepend_with_to_stmnt=True)
# print(usg_kwh_net_or_total_sql_stmnt)
#-------------------------------
usg_kwh_net_df = pd.read_sql(usg_kwh_net_or_total_sql_stmnt + "\nSELECT * FROM U", conn_aws)
usg_kwh_net_df = clm.remove_table_aliases(usg_kwh_net_df)

In [ ]:
usg_norm_df_combined = AMINonVee.combine_kwh_delivered_and_received_values(usg_norm_df)
cols_to_comp = [x for x in usg_kwh_net_df if x!='aep_srvc_qlty_idntfr']
sort_by = ['aep_endtime_utc', 'serialnumber']
usg_norm_df_combined[cols_to_comp].sort_values(by=sort_by, ignore_index=True).equals(usg_kwh_net_df[cols_to_comp].sort_values(by=sort_by, ignore_index=True))

In [ ]:
usg_norm_df['aep_srvc_qlty_idntfr'].unique()

In [ ]:
# usg_norm_df_combined.shape should equal usg_kwh_net_df.shape
# usg_norm_df.shape[0] should be 2x the others, as in the others
#   DELIVERED/RECIEVED or TOTAL/RECEIVED have been combined into CALCULATED_NET(or TOTAL)

In [ ]:
print(f'usg_norm_df.shape          = {usg_norm_df.shape}')
print(f'usg_norm_df_combined.shape = {usg_norm_df_combined.shape}')
print(f'usg_kwh_net_df.shape       = {usg_kwh_net_df.shape}')

In [ ]:
usg_norm_df.sort_values(by=['aep_endtime_utc', 'serialnumber', 'aep_srvc_qlty_idntfr']).head()

In [ ]:
usg_kwh_net_df.sort_values(by=['aep_endtime_utc', 'serialnumber', 'aep_srvc_qlty_idntfr']).head()

In [ ]:
# TODO WHY do multiple entries have TOTAL=RECEIVED????? (when using tmp_sns)
usg_norm_df[usg_norm_df['aep_endtime_utc']=='1609478100'].sort_values(by='serialnumber')

# Looking into weirdness
- This pre-dates the SQLAids package, and AEP was switching to AWS during the time these data were acquired.  Therefore, probably don't worry too much about this weirdness.

In [ ]:
df_old = pd.read_csv(os.path.join(Utilities.get_local_data_dir(), r'sample_circuits\OldMethod\outg_rec_nb_11751094\outg_rec_nb_11751094_2020_q1.csv'))
df_new = pd.read_csv(os.path.join(Utilities.get_local_data_dir(), r'sample_circuits\NewMethod\outg_rec_nb_11751094\outg_rec_nb_11751094_2020_q1.csv'))
df_xfr = pd.read_csv(os.path.join(Utilities.get_local_data_dir(), r'sample_circuits\NewMethod\GroupByXfmr\outg_rec_nb_11751094\outg_rec_nb_11751094_2020_q1.csv'))
df_batch1 = pd.read_csv(os.path.join(Utilities.get_local_data_dir(), r'sample_circuits\Batch1_MoreCounts\outg_rec_nb_11751094\outg_rec_nb_11751094_2020_q1.csv'))
df_batch2 = pd.read_csv(os.path.join(Utilities.get_local_data_dir(), r'sample_circuits\Batch2_LessCounts\outg_rec_nb_11751094\outg_rec_nb_11751094_2020_q1.csv'))

In [ ]:
df_old = clm.remove_table_aliases(df_old)
df_new = clm.remove_table_aliases(df_new)
df_xfr = clm.remove_table_aliases(df_xfr)
df_batch1 = clm.remove_table_aliases(df_batch1)
df_batch2 = clm.remove_table_aliases(df_batch2)

In [ ]:
print(f'df_old.shape    = {df_old.shape}')
print(f'df_new.shape    = {df_new.shape}')
print(f'df_xfr.shape    = {df_xfr.shape}')
print(f'df_batch1.shape = {df_batch1.shape}')
print(f'df_batch2.shape = {df_batch2.shape}')

### Take circuit w/ transformer (df_xfr) to circuit (df_new)

In [ ]:
gpby_cols = [
    'starttimeperiod',
    'endtimeperiod',
    'aep_endtime_utc',
    'timezoneoffset',
    'aep_derived_uom',
    'aep_srvc_qlty_idntfr',
    'aep_usage_dt'
]
sort_cols = ['aep_endtime_utc', 'aep_derived_uom', 'aep_srvc_qlty_idntfr'] #Sort for comparison

In [ ]:
df_xfr_gpd = df_xfr.groupby(gpby_cols).agg({'value_sum':'sum', 
                                            'value_sq_sum':'sum', 
                                            'counts':'sum', 
                                            'counts_including_null':'sum'})
df_xfr_gpd=df_xfr_gpd.reset_index()
df_xfr_gpd['value_mean'] = df_xfr_gpd['value_sum']/df_xfr_gpd['counts']
df_xfr_gpd['value_std'] = np.sqrt((df_xfr_gpd['value_sq_sum'] - df_xfr_gpd['value_sum']*df_xfr_gpd['value_sum']/df_xfr_gpd['counts'])/(df_xfr_gpd['counts']-1))

In [ ]:
df_xfr_gpd = df_xfr_gpd[df_new.columns]
df_xfr_gpd = df_xfr_gpd.sort_values(by=sort_cols, ignore_index=True)
df_new = df_new.sort_values(by=sort_cols, ignore_index=True)

In [ ]:
print(df_xfr_gpd.equals(df_new))
diffs = Utilities_df.get_dfs_diff_approx_ok(df_xfr_gpd, df_new)
print(diffs)

# Why are batch1 and batch2 so much larger than current runs?

In [ ]:
sort_cols = ['aep_endtime_utc', 'aep_derived_uom', 'aep_srvc_qlty_idntfr'] #Sort for comparison
df_new = df_new.sort_values(by=sort_cols, ignore_index=True)
df_batch1 = df_batch1.sort_values(by=sort_cols, ignore_index=True)
df_batch2 = df_batch2.sort_values(by=sort_cols, ignore_index=True)

In [ ]:
df_batch1.head()

In [ ]:
df_new.head()

In [ ]:
df_batch2.head()